In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

from pathlib import Path

In [3]:
cwd =  Path.cwd().parent

folder = cwd /'data'
ibactivity = folder/'ibactivity_processed'
tables = cwd / 'data'/'tables'
tables

WindowsPath('c:/Users/Javier/Python/PortfolioPython/code/data/tables')

In [4]:
# Date Ranges for SP 500

start_sp = datetime(2019, 2, 13)
end_sp =  datetime(2019, 10, 2)
# The below will pull back stock SP500 Total Return.

yf.pdr_override() # <== that's all it takes :-)
sp500 = pdr.get_data_yahoo('^SP500TR', start_sp, end_sp)
#Resample by month, quarter, year ...:
#sp500 = sp500.resample('BM').last() #Uncoment one
#No resample eg daily:
sp500 = sp500.copy() #Uncoment one
start_sp, end_sp

[*********************100%***********************]  1 of 1 downloaded


(datetime.datetime(2019, 2, 13, 0, 0), datetime.datetime(2019, 10, 2, 0, 0))

In [5]:
def get_prices(symbol, first_date, last_date):
    yf.pdr_override()
    prices = pdr.get_data_yahoo(symbol, first_date, last_date)
    prices = prices[['Close']]
    return prices 

In [6]:
files = []
paths = list (sorted(ibactivity.glob('*.csv')))
for p in paths:
    p = p.name.strip('.csv')
    p = p.split('_')
    p = p[-1]
    p = datetime.strptime(p,'%Y%m%d' )
    files.append(p)
#files[0].strip('.csv')
#files = files[0].split('_')
last_report = files[-1]
last_report

datetime.datetime(2019, 10, 31, 0, 0)

In [7]:
#Stock list
# Stock that currently holded
stock_list = pd.read_csv(tables / 'tables_trades.csv')

stock_list = stock_list.loc[stock_list['Asset Category']== 'Stocks']
stock_list = stock_list[['Date/Time','Symbol','Quantity_Rsum']]
stock_list['Date/Time'] = pd.to_datetime(stock_list['Date/Time']).dt.normalize()
#stock_list = stock_list.loc[stock_list['Symbol']== 'POPD']
stock_list.sort_values(by='Date/Time', ascending=True, inplace=True)
stock_list['last_report'] = last_report


### Update ranges
Find if trades_prices exists. 
find last update.
else last update = 01/01/17

stocks trades groupby symbol
Assign min_date to first_trade variable
If Quantity_Rsum = 0 max_date = last_trade
Status close
else status open







Example 700
list = empty list
lst_report = 15/02/20
first_trade = 27/02/18
last_trade  = 27/02/18
last_update = 01/10/19
Quantity_Rsum > 0 
status open


if tables_prices is file:
    if status is closed:
        if last_trade < lst_report
            skip symbol
        else: #last_trade > lst_report
            start = 01/10/19
            end = last_trade
    else: #Open
        last_price = 01/10/19 # last_update
        start = 02/01/19
        end = 15/02/20

else: #tables_prices not file
    start = 27/02/18 #first_trade
    end = 15/02/20  #last_update
Append to list Symbol start end




In [8]:
if (tables/'tables_prices.csv').is_file():
    df = pd.read_csv(tables/'tables_prices.csv')
    last_price_date = df['Date'].max()
else:
    last_price_date = datetime.strptime('20170101','%Y%m%d' )

In [9]:

trades_date= pd.DataFrame()
trades_date['first_trade'] = stock_list.groupby('Symbol').min()['Date/Time']
trades_date['last_price'] = last_price_date

trades_date['First'] = np.where(trades_date['first_trade'] > trades_date['last_price'], trades_date['first_trade'], trades_date['last_price'])

trades_date['last_trade'] = stock_list.groupby('Symbol').max()['Date/Time']
trades_date['last_report'] = stock_list.groupby('Symbol').max()['last_report']
trades_date['State'] =np.where(stock_list.groupby('Symbol').last()['Quantity_Rsum'] == 0,'Closed','Open')



trades_date['Skip'] = np.where((trades_date['State'] == 'Closed') &  (trades_date['last_price'] > trades_date['last_trade']),True,False)

trades_date['First'] = np.where(trades_date['first_trade'] > trades_date['last_price'], trades_date['first_trade'], trades_date['last_price'])

trades_date['Last'] = np.where(trades_date['State'] == 'Open', trades_date['last_report'], trades_date['last_trade'])

trades_date = trades_date.loc[trades_date.Skip == False]

trades_date = trades_date[['First','Last']]
trades_date.reset_index(inplace=True)

values = trades_date.to_dict('records')

In [10]:
tst = values[2]
print(tst['Symbol'])
prices = get_prices('1COV.DE', tst['First'], tst['Last'])
print(prices)


1COVd
[*********************100%***********************]  1 of 1 downloaded
            Close
Date             
2019-06-03  39.29
2019-06-04  41.18
2019-06-05  40.84
2019-06-06  41.08
2019-06-07  41.37
...           ...
2019-10-24  46.74
2019-10-25  46.69
2019-10-28  45.34
2019-10-29  44.84
2019-10-30  43.44

[106 rows x 1 columns]


In [11]:
# prices = get_prices(tst['Symbol'], tst['First'], tst['Last'])
# prices

In [12]:
trades_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 3 columns):
Symbol    85 non-null object
First     85 non-null datetime64[ns]
Last      85 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 2.1+ KB


In [13]:
#end_sp
#if tables_prices is file


In [14]:
#function